# Examples of Continuous Wavelet

Author: Patrice Fortin

Date: 2025-01-06

The following Jupyter Notebook gives code examples for inspecting the internals of the Wavelet Transform Coherence (WTC) by showing figures of different steps of the computation.

The figures can be used to visually inspect the differences between choices of parameters, and libraries.

HyPyP is expected to use Pywavelets as a default implementation. It is the recommended library to use for wavelet in python, and the one that offers the most flexibility. The Pywavelets library is the only one expected to be used in production code.

But, for the sake of validating the HyPyP implementation of WTC, 4 different libraries have been explored.


| Library | Runtime | Details |
| - | - | - |
| Pywavelets | python | The default. Should always be used for production code |
| Pycwt | python | Based on matlab's code |
| Scipy | python | Deprecated, without optimisation so easier to understand the library's code |
| WaveletCoherence | matlab | Calls matlab engine |



## Load libraries


In [ ]:
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
%load_ext IPython.extensions.autoreload
%autoreload 2

# To set breakpoints in a dependency library, use the following strategy
#import sys
#sys.path.insert(0, '/path/to/pycwt')
#import pycwt

import hypyp.fnirs as fnirs
from hypyp.utils import Task
from hypyp.wavelet import ComplexMorletWavelet, ComplexGaussianWavelet, CWT, CoherenceDataFrame
# import other implementations
from hypyp.wavelet.implementations.pycwt_wavelet import PycwtWavelet

## Download and load raw data from disk

To use as example, we download the dataset "Dataset of parent-child hyperscanning fNIRS recordings" from https://researchdata.ntu.edu.sg/dataset.xhtml?persistentId=doi:10.21979/N9/35DNCW

In [ ]:
browser = fnirs.DataBrowser()
browser.download_demo_dataset()


Prepare a list of file paths to be used, without distinction of dyads or parent/child. For this example we just want some data.


In [ ]:
# Get the paths of files

paths = [path for path in browser.list_all_files() if 'fathers' in path]

pattern = re.compile(r'.*/(parent|child)/.*')

file_paths = []
for path in paths:
    if pattern.match(path) is not None:
        file_paths.append(path)
    
if len(file_paths) == 0:
    raise RuntimeError('Cannot find a fNIRS file')

file_path = file_paths[0]

#print(file_path)
    

## Define the wavelets

We prepare a list of wavelets to be compared. We set the same period_range for every wavelet.

Disabling the caching on wavelets is not mandatory here, but in the view that this code serves as an example, if one modifies this and reuse the same wavelet in different places without considering the fact that it caches results, one might see unexpected behaviour.

In [ ]:
wavelet_kwargs = dict(disable_caching=True, period_range=(1, 30))

wavelets = [
    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=0.4, **wavelet_kwargs),
    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=0.7, **wavelet_kwargs),
    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=1, **wavelet_kwargs), # default
    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=2, **wavelet_kwargs),

    ComplexGaussianWavelet(degree=2, **wavelet_kwargs),
    ComplexGaussianWavelet(degree=3, **wavelet_kwargs),
    ComplexGaussianWavelet(degree=4, **wavelet_kwargs),
    ComplexGaussianWavelet(degree=5, **wavelet_kwargs),

]



Prepare a python iterator to easily loop over each wavelet and have a handy ax object for grid plotting.

In [ ]:
def wavelets_plot_iterator():
    cols = 4
    rows = int(np.ceil(len(wavelets) / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5*rows))
    axes = np.atleast_2d(axes)
    for i, wavelet in enumerate(wavelets):
        row = i // cols
        col = i % cols
        ax = axes[row][col]
        yield wavelet, ax


### Mother wavelets

Let's compare mother wavelets. The x axis differs between libraries because of implementation details.

In [ ]:

for wavelet, ax in wavelets_plot_iterator():
    wavelet.plot_mother_wavelet(ax=ax)
    ax.set_title(f'[{wavelet.wavelet_library}] {wavelet.wavelet_name}')
_ = ax.get_figure().suptitle('Mother Wavelet for various libraries and parameters')


### Form the dyad and channel pairs

Prepare the signals to be used in the Continuous Wavelet Transforms. We use the first 2 files and compare the pair of the first channel in both subject

In [ ]:
tasks = [Task('sample', onset_time=60, duration=60)]
subject1 = fnirs.Subject(label='Subject1', tasks=tasks).load_file(file_paths[0], fnirs.MnePreprocessorRawToHaemo())
subject2 = fnirs.Subject(label='Subject2', tasks=tasks).load_file(file_paths[1], fnirs.MnePreprocessorRawToHaemo())
dyad = fnirs.Dyad(subject1, subject2, label=f'intra')
# take one pair
pair = dyad.get_pairs(subject1, subject2)[0]


Visualize the 2 signals


In [ ]:
plt.plot(pair.x, pair.y1)
plt.plot(pair.x, pair.y2)
plt.title('Signals pair')
plt.show()


# Continuous Wavelet Transform (CWT)

We run the Continuous Wavelet Transform on both channels of the pair and visually inspect the results for every wavelet choices.

The darker area correspond to the Cone of Influence, for which periods greater than a certain value are more subject to artifacts. 

**NOTE: The computation of the Cone of Influence in the pywavelets should be reviewed for validity.**


In [ ]:
for wavelet, ax in wavelets_plot_iterator():
    cwt = wavelet.cwt(pair.y1, pair.dt)
    cwt.plot(ax=ax, show_colorbar=False)
    ax.set_title(f'CWT [{wavelet.wavelet_library}] {wavelet.wavelet_name}')
    ax.get_figure().suptitle('Continuous Wavelet Transform of subject1, channel1')
plt.show()

for wavelet, ax in wavelets_plot_iterator():
    cwt = wavelet.cwt(pair.y2, pair.dt)
    cwt.plot(ax=ax, show_colorbar=False)
    ax.set_title(f'CWT [{wavelet.wavelet_library}] {wavelet.wavelet_name}')
    ax.get_figure().suptitle('Continuous Wavelet Transform of subject2, channel1')
plt.show()

## Compare smoothed CWTs


We explore the smooting of the the CWT. The weights of the CWT are smooted both in time and in frequency. 

The code below replicates the implementation details in the `BaseWavelet` class.

In [ ]:
for wavelet, ax in wavelets_plot_iterator():
    cwt = wavelet.cwt(pair.y1, pair.dt)

    scales = np.ones([1, len(pair.y1)]) * cwt.scales[:, None]
    smooted = wavelet.smoothing(np.abs(cwt.W) ** 2 / scales, cwt.dt, cwt.scales)
    cwt_smoothed = CWT(smooted**2, cwt.times, cwt.scales, cwt.periods, cwt.coi)
    cwt_smoothed.plot(ax=ax, show_colorbar=False)
    ax.set_title(f'Smoothed CWT [{wavelet.wavelet_library}] {wavelet.wavelet_name}')
    ax.get_figure().suptitle('Smoothed Continuous Wavelet Transform of subject 1, channel 1')
plt.show()



In [ ]:
for wavelet, ax in wavelets_plot_iterator():
    cwt = wavelet.cwt(pair.y2, pair.dt)

    scales = np.ones([1, len(pair.y2)]) * cwt.scales[:, None]
    smooted = wavelet.smoothing(np.abs(cwt.W) ** 2 / scales, cwt.dt, cwt.scales)
    cwt_smoothed = CWT(smooted**2, cwt.times, cwt.scales, cwt.periods, cwt.coi)
    cwt_smoothed.plot(ax=ax, show_colorbar=False)
    ax.set_title(f'Smoothed CWT [{wavelet.wavelet_library}] {wavelet.wavelet_name}')
    ax.get_figure().suptitle('Smoothed Continuous Wavelet Transform of subject 2, channel 1')
plt.show()




## Compare Wavelet Transform Coherence (WTC)

Now that we can trust our CWT and smoothing, we compare the WTC results for each wavelet.

In [ ]:
for wavelet, ax in wavelets_plot_iterator():
    wtc = wavelet.wtc(pair)
    wtc.plot(ax=ax, show_colorbar=False)
    ax.set_title(f'WTC [{wavelet.wavelet_library}] {wavelet.wavelet_name}')


## Time and frequency segmentation

When doing statistical analysis, multiple recordings of dyads are necessary. For each recording, a set of tasks are usually given, and can be recovered by events in the fNIRS recording file. 

These tasks may be recurring tasks, and have a different duration each time. A robust analysis should weight each recording by its duration.

Also, recordings can present a number of signal issues, either motion artifacts that cannot be filtered, or a connectivity failing on a specific channel, to name a few. To avoid excluding channels with punctual invalid data, a signal on a channel can be splitted in segments and the WTC will be computed to the matching segment of other channels. This will result of multiple segments of smaller duration. We also need to weight accordingly.

It can also be useful to split the resulting data into periods slices, to find the most "coherent" period ranges.

In order to have a balanced weight and period slices, once the WTC has been computed on a channel pair, we can split the WTC matrix in a grid.

The figure below explicitly shows the grid for a computed WTC.

In [ ]:
wavelet = ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=1)
bin_seconds = 10
period_cuts = [5, 10]
dyad = fnirs.Dyad(subject1, subject2)
dyad.compute_wtcs(wavelet=wavelet, ch_match='S1_D1 hbo', bin_seconds=bin_seconds, period_cuts=period_cuts, verbose=False)

wtc = dyad.wtcs[0]
fig = wtc.plot(
    show_bins=True,
    title=f'Grid pattern with bin_seconds={bin_seconds}, period_cuts={period_cuts}',
)




We can also plot the coherence as time series, using a moving average

In [ ]:

window_size = 10
time_series = wtc.get_as_time_series(window_size=window_size)
for i in range(time_series.shape[0]):
    ts = time_series[i,:]
    plt.plot(wtc.times, ts, label=str(wtc.p_ranges_str[i]))
    plt.title(f"Coherence Time Series (window size: {window_size/wtc.sfreq:.1f}s)")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Coherence")
plt.ylim(0, 1)
plt.show()


Each bin correspond to one row in the resulting pandas dataframe. Bins with more than 50% of their area in the cone of influence will be excluded by setting `NaN` in the coherence. The `coherence_masked` correspond to the ratio of area that is subject to artifacts

In [ ]:
dyad.df[['bin', 'coherence', 'coherence_masked', 'bin_time_range', 'bin_period_range']]


# Coherence on multiple channel pairs

## Exploring the impact of Wavelet parameters on coherence

It is interesting to note that the current litterature on fNIRS hyperscanning uses a set of parameters for computing the Wavelet Transform Coherence (WTC) that has been established more than 20 years ago in the field of geophysics. The excellent article "A Practical Guide to Wavelet Analysis" from Terrence and Compo (1998) is foundational on the subject.

In order to validate that these parameters are indeed fit for fNIRS Inter Brain Synchrony (IBS) analysis using the WTC strategy analysis, we suggest here a way to explore the impact of these parameters on the IBS research.

Here is an example of how to compare the impact of wavelet parameters on the coherence results.

The code below defines multiple wavelets with different parameters. For each wavelet, it computes the WTC on a list of intra-subjects.

This is an example to serve as a starting point for a real analysis. A complete analysis would compute the WTC on more dyads and compare the statistical significance of the coherence given different set of parameters.

Considered parameters:

* Wavelet family (Complex Morlet and Complex DOG)
* Wavelet center frequency
* Wavelet bandwidth (Morlet only)
* Smoothing window size

There are mathematical imperatives that are not taken into consideration in the following list of wavelet parameters, and we recommend a careful review of wavelet analysis litterature before selecting the parameter pairs and drawing conclusions.


In [ ]:

wavelet_kwargs = dict(period_range=(1, 30))
wavelets = [
    ComplexMorletWavelet(bandwidth_frequency=1, center_frequency=1, **wavelet_kwargs),
    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=1, **wavelet_kwargs),
    ComplexMorletWavelet(bandwidth_frequency=3, center_frequency=1, **wavelet_kwargs),
    ComplexMorletWavelet(bandwidth_frequency=4, center_frequency=1, **wavelet_kwargs),

    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=0.5, **wavelet_kwargs),
    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=0.8, **wavelet_kwargs),
    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=1, **wavelet_kwargs),
    ComplexMorletWavelet(bandwidth_frequency=2, center_frequency=1.5, **wavelet_kwargs),

    ComplexMorletWavelet(wtc_smoothing_win_size=0.001, **wavelet_kwargs),
    ComplexMorletWavelet(wtc_smoothing_win_size=0.1, **wavelet_kwargs),
    ComplexMorletWavelet(wtc_smoothing_win_size=0.6, **wavelet_kwargs),
    ComplexMorletWavelet(wtc_smoothing_win_size=1, **wavelet_kwargs),

    ComplexGaussianWavelet(degree=2, **wavelet_kwargs),
    ComplexGaussianWavelet(degree=3, **wavelet_kwargs),
    ComplexGaussianWavelet(degree=4, **wavelet_kwargs),
    ComplexGaussianWavelet(degree=5, **wavelet_kwargs),
]

dyads = []
tasks = [Task('sample', onset_time=60, duration=60)]
processor = fnirs.MnePreprocessorRawToHaemo()
bin_seconds = 10
period_cuts = [5, 10]
n_dyads = 3
for wavelet in wavelets:
    for i in np.arange(0, n_dyads):
        file_path1 = file_paths[i*2]
        file_path2 = file_paths[i*2+1]
        #print(f'WTC "{wavelet.wavelet_name}" on file "{file_path1}" and {file_path2}')
        # clear previous cache from loop
        wavelet.clear_cache()
        subject1 = fnirs.Subject(tasks=tasks, label='subject1').load_file(file_path1, processor)
        subject2 = fnirs.Subject(tasks=tasks, label='subject2').load_file(file_path2, processor)
        dyad = fnirs.Dyad(subject1, subject2, label=f'{subject1.label}-{subject2.label} [{wavelet.wavelet_library}] {wavelet.wavelet_name}')
        dyad.compute_wtcs(wavelet=wavelet, ch_match='hbo', bin_seconds=bin_seconds, period_cuts=period_cuts, verbose=False)
        dyads.append(dyad)



In [ ]:

df = CoherenceDataFrame.concat([dyad.df for dyad in dyads])

plt.figure(figsize=(12, 12))

df['wavelet_key'] = df['wavelet_library'].astype(str) + ['_']*df.shape[0] + df['wavelet_name'].astype(str)
df['wavelet_key'] = df['wavelet_key'].astype('category')

flierprops = {"marker": ".", "markerfacecolor": "black", "markersize": 4}

sns.boxplot(data=df, y='wavelet_key', x='coherence', hue='bin_period_range', orient='h', flierprops=flierprops)

# Customize the plot
plt.legend(title='Period ranges', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.title('Comparison of coherence')
plt.tight_layout()  # Adjust layout to prevent label cutoff
plt.show()

